In [1]:
"""
@author: cdstst
"""

import numpy as np
from numpy import *

def feature_sign_search(A, y, gamma):

    effective_zero = 1e-18

    AtA = np.dot(A.T, A)
    Aty = np.dot(A.T, y)
    x = np.zeros(AtA.shape[0])
    theta = np.zeros(AtA.shape[0], dtype=np.int8)
    active_set = set()
    z_opt = np.inf
    
    nz_opt = 0
    grad = - 2 * Aty + 2 * np.dot(AtA, x)


    norm_y = np.dot(y.T, y)
    while z_opt > gamma or not np.allclose(nz_opt, 0):
        if np.allclose(nz_opt, 0):
            i = np.argmax(np.abs(grad) * (theta == 0))
            if (grad[i] > gamma):
                theta[i] = -1.
                x[i] = 0.
                active_set.add(i)
            elif grad[i] < -gamma:
                theta[i] = 1.
                x[i] = 0.
                active_set.add(i)
            if len(active_set) == 0:
                break

        act_ind = np.array(sorted(active_set))
        AtA_hat = AtA[np.ix_(act_ind, act_ind)]
        Aty_hat = Aty[act_ind]
        theta_hat = theta[act_ind]
        rhs = Aty_hat - gamma * theta_hat / 2

        new_x = np.linalg.solve(np.atleast_2d(AtA_hat), rhs)
        new_theta = np.sign(new_x)
        restr_oldsol = x[act_ind]
        sign_changes = np.where(abs(new_theta - theta_hat) > 1)[0]
        if len(sign_changes) > 0:
            best_obj_val = np.inf
            best_curr = None
            best_curr = new_x
            best_obj_val = (norm_y + (np.dot(new_x,np.dot(AtA_hat, new_x))- 2 * np.dot(new_x, Aty_hat))+ gamma * abs(new_x).sum())

            for idx in sign_changes:
                a = new_x[idx]
                b = restr_oldsol[idx]
                prop = b / (b - a)
                curr = restr_oldsol - prop * (restr_oldsol - new_x)
                cost = norm_y + (np.dot(curr, np.dot(AtA_hat, curr))- 2 * np.dot(curr, Aty_hat)+ gamma * abs(curr).sum())
                if cost < best_obj_val:
                    best_obj_val = cost
                    best_curr = curr
        else:
            best_curr = new_x
        x[act_ind] = best_curr
        zeros = act_ind[np.abs(x[act_ind]) < effective_zero]
        x[zeros] = 0.
        theta[act_ind] = np.int8(np.sign(x[act_ind]))
        active_set.difference_update(zeros)
        grad = - 2 * Aty + 2 * np.dot(AtA, x)
        z_opt = np.max(abs(grad[theta == 0]))
        nz_opt = np.max(abs(grad[theta != 0] + gamma * theta[theta != 0]))
    return x

In [2]:
#test
mylist1 = []
for i in range(50):
    mylist1.append(np.random.rand(50))
test_A = np.array(mylist1)

mylist2 = np.empty([0,1])
for i in range(50):
    mylist2=np.append(mylist2,np.random.rand(1))
test_y = np.array(mylist2)


test_gamma=0.2

final_sol=feature_sign_search(test_A,test_y,test_gamma)
print (final_sol)

[-0.09054842 -0.13742147  0.          0.22901355  0.00105319  0.09142694
  0.03952626 -0.06874425  0.44230792 -0.00985492 -0.17737639 -0.08563641
  0.04071678  0.19430305  0.13650251 -0.40019362  0.          0.
  0.13254252  0.          0.          0.08057005  0.12539661  0.19236565
 -0.03542179  0.11871915  0.14083261 -0.06018847  0.          0.01811271
  0.0903241   0.01232529  0.          0.11430708  0.24857124  0.12034321
  0.21769047 -0.28800984 -0.02121575 -0.3531502  -0.10801919  0.17663218
  0.20381766  0.         -0.06818087 -0.27042301  0.          0.
 -0.06993176  0.        ]
